In [ ]:
import tensorflow_datasets as tfds

# Load the training split and convert to numpy for easier handling
train_data = tfds.as_numpy(tfds.load('imdb_reviews', split='train', batch_size=-1))
test_data = tfds.as_numpy(tfds.load('imdb_reviews', split='test', batch_size=-1))

# Extract text and labels
train_sentences = [str(x) for x in train_data['text']]
train_labels = train_data['label']

test_sentences = [str(x) for x in test_data['text']]
test_labels = test_data['label']


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.MSZE5E_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.MSZE5E_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.MSZE5E_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define the vocabulary size and sequence length
vocab_size = 10000
max_length = 100
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, maxlen=max_length, truncating='post')

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=max_length, truncating='post')


In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 16, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    train_padded, train_labels,
    epochs=3,
    validation_data=(test_padded, test_labels)
)


Epoch 1/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 57s 67ms/step - accuracy: 0.6321 - loss: 0.6005 - val_accuracy: 0.8033 - val_loss: 0.4208
Epoch 2/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 79s 64ms/step - accuracy: 0.8780 - loss: 0.2974 - val_accuracy: 0.8109 - val_loss: 0.4204
Epoch 3/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 83s 65ms/step - accuracy: 0.9192 - loss: 0.2125 - val_accuracy: 0.8010 - val_loss: 0.5504


In [ ]:
import numpy as np

def predict_sentiment(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_length, truncating='post')
    prediction = model.predict(padded)[0][0]
    sentiment = "😊 Positive" if prediction > 0.5 else "😞 Negative"
    print(f"Review: {text}")
    print(f"Prediction: {sentiment} (score: {prediction:.3f})")

# Try it!
predict_sentiment("This movie was absolutely fantastic! The plot was brilliant.")
predict_sentiment("I hated this movie. It was boring and too long.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step
Review: This movie was absolutely fantastic! The plot was brilliant.
Prediction: 😊 Positive (score: 0.802)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Review: I hated this movie. It was boring and too long.
Prediction: 😞 Negative (score: 0.031)


In [ ]:
while True:
    user_input = input("Enter a movie review (or 'quit' to exit): ")
    if user_input.lower() == 'quit':
        break
    predict_sentiment(user_input)


Enter a movie review (or 'quit' to exit): the batman, one of the masterpiece i saw in recent past. since covid, it has bring the urge to go back to cinema and watch it 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Review: the batman, one of the masterpiece i saw in recent past. since covid, it has bring the urge to go back to cinema and watch it 
Prediction: 😊 Positive (score: 0.943)
Enter a movie review (or 'quit' to exit): quit
